In [2]:
import math
print("hello world")

hello world


Here we will read the dataset and transfer it to the base SQL

In [2]:
file = open("dataset.csv", "r", encoding="utf-8")
output = open("insert_dataset.sql", "w", encoding="utf-8")

# Nombres de columnas en el CSV
columns = [
    "id","product_A_sold_in_the_past","product_B_sold_in_the_past",
    "product_A_recommended","product_A","product_C","product_D",
    "cust_hitrate","cust_interactions","cust_contracts",
    "opp_month","opp_old",
    "competitor_Z","competitor_X","competitor_Y",
    "cust_in_iberia","target_variable"
]

acces = False

for line in file:
    if acces == False:
        acces = True
        continue  # saltar cabecera

    values = line.strip().split(";")

    # Escapar valores de texto con comillas
    escaped = [f"'{v}'" for v in values]

    sql = (
        f"INSERT INTO mi_tabla ({','.join(columns)}) "
        f"VALUES ({','.join(escaped)});\n"
    )

    output.write(sql)

file.close()
output.close()

print("Archivo SQL generado.")


Archivo SQL generado.
